Parsing the Emails (Mailbox format)

subject
Body
Absender
time
past messages

In [1]:
import os
print(os.getcwd())

/home/stud17/python/spam2


In [2]:
import os
import email
from email import policy

def extract_body_from_email(path):
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        msg = email.message_from_file(f)
    
    # If multipart, go through parts
    if msg.is_multipart():
        parts = []
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':  # we only want the readable part
                charset = part.get_content_charset()
                payload = part.get_payload(decode=True)
                if payload is None:
                    continue
                try:
                    text = payload.decode(charset or 'utf-8', errors='ignore')
                except (LookupError, UnicodeDecodeError):
                    text = payload.decode('utf-8', errors='ignore')
                parts.append(text)
        return '\n'.join(parts)
    else:
        # Single-part message
        payload = msg.get_payload(decode=True)
        charset = msg.get_content_charset()
        try:
            return payload.decode(charset or 'utf-8', errors='ignore')
        except (LookupError, UnicodeDecodeError):
            return payload.decode('utf-8', errors='ignore')

test in one email

In [3]:
sample_path = 'data/spam2-train/aahkoctdmjnikvon.0'
print(extract_body_from_email(sample_path))

(Not on the list, please CC me on replies)

hey,
 ia64 cpus have dropped hardware i386 support in exchange for non-free
software emulation, known as "IA-32 EL". I took a cursory look at the
license, and I'm wondering if its even viable to upload to non-free.

The text of this license is available here:
  http://www3.intel.com/cd/software/products/asmo-na/eng/219741.htm

It seems pretty clear that the intent is to permit redistribution, but
I'm unsure about some of the restrictions. I'd like to collect a list
of issues with this license (if any) with respect to non-free and see
if Intel is willing to change them. The one that sticks out to me is:

 * Section 4: "Updates" requires "commerically reasonable efforts" to
   supply our "customers" with updates that Intel distributes. If this
   means we cannot say no to an update from Intel, that does not sound
   reasonable for non-free. But perhaps we are exempt from this since
   1) non-free isn't officially part of Debian and 2) we are a


looks good, doing all and start experimenting

In [4]:
def load_dataset(data_dir, labels_file):
    """
    Loads all emails and their labels into memory.

    Args:
        data_dir (str): Path to the folder containing the email files
        labels_file (str): Path to the labels file (e.g., "spam2-train.labels")

    Returns:
        X (list of str): List of preprocessed email bodies
        y (list of int): List of labels (0 = ham, 1 = spam)
    """
    X, y = [], []

    with open(labels_file, "r", encoding="utf-8") as f:
        for line in f:
            # Each line: "data/spam2-train/filename;label"
            filepath, label = line.strip().split(";")
            label = int(label)

            # Extract just the filename part after the last '/'
            filename = os.path.basename(filepath)

            # Full path to the email file
            email_path = os.path.join(data_dir, filename)

            # Get the clean text body
            text = extract_body_from_email(email_path)

            X.append(text)
            y.append(label)

    return X, y


In [5]:
X, y = load_dataset("data/spam2-train", "spam2-train.labels")

In [ ]:
#test
print(len(X), len(y))
print(X[0][:300])  # preview first email
print(y[0])


18756 18756
Ok, so I see now that reg_alloc is rounded up to a multiple of 8 by
the following two lines:

   /*code*/ const int slot = (reg_alloc + 7) >> 3;
   /*code*/ reg_alloc = slot << 3;

However, this still begs the question of what the slot variable is
for. Clearly it's being used as an index into
interp
0


3. Extract Features and Vectorize the Text

Next, you need to convert the cleaned text into numerical features that can be used by the machine learning model.

TF-IDF Vectorizer: This is the most common method for text data. It transforms the text into a sparse matrix of term frequencies.